In [1]:
import os, re, h5py
import numpy as np
import tifffile as Tiff
from PIL import Image
from skimage import io
from skimage import exposure
from skimage.viewer import ImageViewer
from subprocess import call
from natsort import natsorted
import subprocess
import psycopg2
import warnings; warnings.simplefilter('ignore')

/Users/OGN/anaconda/envs/py36/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


### Import HDF5 contents from the Randlett, Burgess, and Heisemeyer lines (Anatomy, ZBB, MH)


In [27]:
# from IPython.display import Image

HDF5s = []
HDF5s.append(h5py.File('AnatomyLabelDatabase.hdf5', 'r'))
HDF5s.append(h5py.File('ZBBDatabase.hdf5', 'r'))
HDF5s.append(h5py.File('Harvard_MH_Lines.hdf5', 'r'))

#### Print the names of all lines (stacks)


In [28]:
print('Anatomy:\n', [k for k in HDF5s[0].keys()], '\n')
print('ZBB:\n', [k for k in HDF5s[1].keys()], '\n')
print('MH:\n', [k for k in HDF5s[2].keys()], '\n')

Anatomy:
 ['6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish', 'Anti-5HT_6dpf_MeanImageOf40Fish', 'Anti-Gad67_6dpf_MeanImageOf17Fish', 'Anti-GlyR_6dpf_MeanImageOf160Fish', 'Anti-TH_6dpf_MeanImageOf10Fish', 'Anti-Zn12(Hnk-1)_6dpf_MeanImageOf9Fish', 'Anti-Zn1_6dpf_MeanImageOf10Fish', 'Anti-Znp1(Synaptotagmin2)_6dpf_MeanImageOf118Fish', 'Anti-Zrf1(GFAP)_6dpf_MeanImageOf5Fish', 'Anti-Zrf2_6dpf_MeanImageOf5Fish', 'Anti-tERK_6dpf_MeanImageOf193Fish', 'Elavl3-GCaMP5G_6dpf_MeanImageOf7Fish', 'Elavl3-H2BRFP_6dpf_MeanImageOf10Fish', 'EtVmat2-GFP_6dpf_MeanImageOf55Fish', 'Gad1b-GFP_6dpf_MeanImageOf10Fish', 'Glyt2-GFP_6dpf_MeanImageOf13Fish', 'Hcrt-RFP_6dpf_MeanImageOf15Fish', 'Isl1-GFP_6dpf_MeanImageOf17Fish', 'Isl2bGal4-uasDendra_6dpf_MeanImageOf8Fish', 'Olig2-GFP_6dpf_MeanImageOf12Fish', 'Otpb.A-Gal4-UAS-GCaMP_6-7dpf_MeanImageOf19Fish', 'Otpb.A-Nsfb-GFP_6dpf_MeanImageOf10Fish', 'Oxtl-GFP_6dpf_MeanImageOf68Fish', 'Pet1-GFP_6dpf_MeanImageOf13Fish', 'Ptf1aGal4-uasKaede_6dpf_MeanImageOf11Fish', 'Qrf

#### Copy single stack from one HDF5

In [5]:
data = HDF5s[0].get('Otpb.A-Nsfb-GFP_6dpf_MeanImageOf10Fish')

#### Copy single slice from one stack

In [6]:
image = data[75]
# tiff.imsave('Otpb_a-Nsfb75.tiff', image)
# io.imsave('Otpb_a-Nsfb75.jpeg', image)

#### Check brightness stats (all images grayscale)

In [7]:
print(np.mean(data))
np.max(data)

4645.82173008


65535

In [8]:
data = HDF5s[1].get('ZBB_y256-Gal4')
print(np.mean(data))

92.4220285768


### Test basic image manipulation with Subprocess >> Imagemagick c.l. tools

In [14]:
adj_data0 = data[90]
try:
    io.imsave('Test/test_16bit_jpeg0.jpg', adj_data0)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'Test/test_16bit_jpeg0.jpg'


#### Image created ^ Now modify with 'convert'

In [15]:
try:
    subprocess.check_output(["convert", "Test/test_16bit_jpeg0.jpg", "-format", "jpg", "-level", "0%, 50%", "Test/test_16bit_jpeg1.jpg"], stderr=subprocess.STDOUT)
except Exception as e:
    print(e)

Command '['convert', 'Test/test_16bit_jpeg0.jpg', '-format', 'jpg', '-level', '0%, 50%', 'Test/test_16bit_jpeg1.jpg']' returned non-zero exit status 1.


#### Display image

In [16]:
try:
    Image("Test/test_16bit_jpeg0.jpg")
except Exception as e:
    print(e)

'module' object is not callable


### Export images from HDF5 files

#### Export tiff stacks from HDF5 files

In [29]:
for h in HDF5s:
    for k, key in enumerate(h.keys()):
        stack = h.get(key)
        stack_name = str(re.split('_6(.*)dpf', key)[0]) # applies only to Anatomy and MH lines, chops dpf suffix
        
        try:
            int(stack_name)                             # applies only to MH Lines
            stack_name = 'MH_{}-Gal4'.format(stack_name)
        except:
            pass
        
        tiff_path = '../app/assets/images/4-TiffStacks/{}.tif'.format(stack_name) # assuming cwd is db/
        if not os.path.isfile(tiff_path):
            print(tiff_path)
            Tiff.imsave(tiff_path, stack)
        

../app/assets/images/4-TiffStacks/MH_13319-Gal4.tif
../app/assets/images/4-TiffStacks/MH_15576-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16659-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16715-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16913-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16914-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16915-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16916-Gal4.tif
../app/assets/images/4-TiffStacks/MH_16946-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17056-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17767-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17768-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17776-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17777-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17787-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17845-Gal4.tif
../app/assets/images/4-TiffStacks/MH_17846-Gal4.tif
../app/assets/images/4-TiffStacks/MH_18107-Gal4.tif
../app/assets/images/4-TiffStacks/MH_18108-Gal4.tif
../app/asset

In [30]:
f = HDF5s[0]
f2 = HDF5s[1]
f3 = HDF5s[2]

#### Export line JPGs from Anatomy HFD5 without creating database entries

In [252]:
for k, key in enumerate(f.keys()[:]):
    stack = f.get(key)
    stack_name = key.split('_')[0]
    folder_path = '../app/assets/images/%s/' % (stack_name)
    print folder_path
    print np.mean(stack)

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        print exception
        
    for i, img in enumerate(stack):
        img_path = '../app/assets/images/%s/%s-%i.jpg' % (stack_name, key, i)
        io.imsave(img_path, stack[i])
        subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%, 50%", img_path], stderr=subprocess.STDOUT)


../app/assets/images/6.7FRhcrtR-Gal4-uasKaede/
2097.70344932
[Errno 17] File exists: '../app/assets/images/6.7FRhcrtR-Gal4-uasKaede/'
../app/assets/images/Anti-5HT/
5949.16077933
[Errno 17] File exists: '../app/assets/images/Anti-5HT/'
../app/assets/images/Anti-Gad67/
3533.79462989
[Errno 17] File exists: '../app/assets/images/Anti-Gad67/'


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-Gad67/Anti-Gad67_6dpf_MeanImageOf17Fish-136.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-Gad67/Anti-Gad67_6dpf_MeanImageOf17Fish-137.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


../app/assets/images/Anti-GlyR/
927.020210507
[Errno 17] File exists: '../app/assets/images/Anti-GlyR/'


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-GlyR/Anti-GlyR_6dpf_MeanImageOf160Fish-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-GlyR/Anti-GlyR_6dpf_MeanImageOf160Fish-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-GlyR/Anti-GlyR_6dpf_MeanImageOf160Fish-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Anti-GlyR/Anti-GlyR_6dpf_MeanImageOf160Fish-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images

../app/assets/images/Anti-TH/
3336.02535204
[Errno 17] File exists: '../app/assets/images/Anti-TH/'


...

/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/S1181tGal4-uasKaede/S1181tGal4-uasKaede_6dpf_MeanImageOf14Fish-136.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/S1181tGal4-uasKaede/S1181tGal4-uasKaede_6dpf_MeanImageOf14Fish-137.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


../app/assets/images/SpinalBackfills/
1683.52242201
[Errno 17] File exists: '../app/assets/images/SpinalBackfills/'


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/SpinalBackfills/SpinalBackfills_6dpf_MeanImageOf23Fish-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/SpinalBackfills/SpinalBackfills_6dpf_MeanImageOf23Fish-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/SpinalBackfills/SpinalBackfills_6dpf_MeanImageOf23Fish-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/SpinalBackfills/SpinalBackfills_6dpf_MeanImageOf23Fish-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_

../app/assets/images/Vglut2a-GFP/
2169.33978154
[Errno 17] File exists: '../app/assets/images/Vglut2a-GFP/'


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Vglut2a-GFP/Vglut2a-GFP_6dpf_MeanImageOf15Fish-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Vglut2a-GFP/Vglut2a-GFP_6dpf_MeanImageOf15Fish-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Vglut2a-GFP/Vglut2a-GFP_6dpf_MeanImageOf15Fish-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/Vglut2a-GFP/Vglut2a-GFP_6dpf_MeanImageOf15Fish-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/a

#### Export line JPGs from ZBB HFD5 without creating database entries

In [266]:
for k, key in enumerate(f2.keys()[:]):
    stack = f2.get(key)
    stack_name = key
    folder_path = '../app/assets/images/%s/' % (stack_name)
    print folder_path
    mean_intensity = np.mean(stack)
    
    if mean_intensity >= 1000:
        white_point = 50
    elif mean_intensity >= 100 and mean_intensity < 1000:
        white_point = 40
    else:
        white_point = 30

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        print exception
    
    print white_point
    for i, img in enumerate(stack):
        img_path = '../app/assets/images/%s/%s-%i.jpg' % (stack_name, key, i)
        #io.imsave(img_path, stack[i])
        #subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%%, %i%%" % (white_point) , img_path], stderr=subprocess.STDOUT)    
        

../app/assets/images/ZBB_HuC-Cer/
[Errno 17] File exists: '../app/assets/images/ZBB_HuC-Cer/'
50
../app/assets/images/ZBB_ath5-GFP/
[Errno 17] File exists: '../app/assets/images/ZBB_ath5-GFP/'
40


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_ath5-GFP/ZBB_ath5-GFP-126.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_ath5-GFP/ZBB_ath5-GFP-127.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_ath5-GFP/ZBB_ath5-GFP-128.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_ath5-GFP/ZBB_ath5-GFP-129.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_ath5-GFP/ZBB_ath5-GFP-130.jpg is a low contrast image
  war

../app/assets/images/ZBB_bActin-GFP/
[Errno 17] File exists: '../app/assets/images/ZBB_bActin-GFP/'
50


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_bActin-GFP/ZBB_bActin-GFP-137.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


../app/assets/images/ZBB_chx10-gal4/
[Errno 17] File exists: '../app/assets/images/ZBB_chx10-gal4/'
40


...

/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y383-Cre/ZBB_y383-Cre-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y383-Cre/ZBB_y383-Cre-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y383-Cre/ZBB_y383-Cre-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y383-Cre/ZBB_y383-Cre-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y383-Cre/ZBB_y383-Cre-4.jpg is a low contrast image
  warn('%s is a

../app/assets/images/ZBB_y384-Cre/
[Errno 17] File exists: '../app/assets/images/ZBB_y384-Cre/'
40


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y384-Cre/ZBB_y384-Cre-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y384-Cre/ZBB_y384-Cre-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y384-Cre/ZBB_y384-Cre-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y384-Cre/ZBB_y384-Cre-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y384-Cre/ZBB_y384-Cre-4.jpg is a low contrast image
  warn('%s is a

../app/assets/images/ZBB_y385-Cre/
[Errno 17] File exists: '../app/assets/images/ZBB_y385-Cre/'
40


/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y385-Cre/ZBB_y385-Cre-0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y385-Cre/ZBB_y385-Cre-1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y385-Cre/ZBB_y385-Cre-2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y385-Cre/ZBB_y385-Cre-3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/OGN/anaconda/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: ../app/assets/images/ZBB_y385-Cre/ZBB_y385-Cre-4.jpg is a low contrast image
  warn('%s is a

#### Export line JPGs from Harvard_MH HFD5 without creating database entries

In [39]:
for k, key in enumerate(f3.keys()):
    stack = f3.get(key)
    stack_number = key.split('_')[0]
    stack_name = 'MH_{}-Gal4'.format(stack_number)
    folder_path = '../app/assets/images/0-Lines/%s/' % (stack_name)
    mean_intensity = np.mean(stack)
    print (stack_name, mean_intensity)

    try:
        os.makedirs(folder_path)
    except OSError as exception:
        pass
    
    white_point = 40
        
    for i, t in enumerate(stack):
        img_path = '{}/{}-{}.jpg'.format(folder_path, stack_name, i)
        io.imsave(img_path, t)
        subprocess.check_output(["convert", img_path, "-format", "jpg", "-level", "0%%, %i%%" % (white_point) , img_path], stderr=subprocess.STDOUT)

MH_13319-Gal4 2812.68795899
MH_15576-Gal4 3574.26017525
MH_16659-Gal4 11615.5238966
MH_16715-Gal4 9433.42074963
MH_16913-Gal4 10686.3600645
MH_16914-Gal4 1450.09333567
MH_16915-Gal4 6869.44971984
MH_16916-Gal4 2636.40262084
MH_16946-Gal4 5873.45485924
MH_17056-Gal4 3005.27221835
MH_17767-Gal4 2076.90699749
MH_17768-Gal4 2019.05356542
MH_17776-Gal4 6648.48493657
MH_17777-Gal4 3427.17666903
MH_17787-Gal4 4313.95343514
MH_17845-Gal4 2636.49697203
MH_17846-Gal4 4107.92464891
MH_18107-Gal4 3221.95892779
MH_18108-Gal4 2774.80982257
MH_18327-Gal4 5405.86273808
MH_18329-Gal4 3135.75183934
MH_19322-Gal4 5112.56925594
MH_19566-Gal4 3466.57842414
MH_20360-Gal4 2407.11234121
MH_20361-Gal4 2513.67460918
MH_20454-Gal4 1228.75951744
MH_20552-Gal4 4935.75330039
MH_20553-Gal4 1847.23119936
MH_20638-Gal4 2387.89796152
MH_20639-Gal4 1210.14545137
MH_20640-Gal4 1347.12983667
MH_20641-Gal4 1341.35001877
MH_20642-Gal4 1790.26426982
MH_20643-Gal4 2817.13097499
MH_20644-Gal4 3250.35008935
MH_20646-Gal4 2192.1

#### Export line JPGs from Reference-Brain tif

In [60]:
tiff = Tiff.imread('..app/assets/images/4-TiffStacks/Reference-Brain.tif')
print(tiff.shape)
tiff = np.squeeze(tiff, axis=0)
tiff = np.squeeze(tiff, axis=1)
print(tiff.shape)

for i, image in enumerate(tiff):
    img_path = '../app/assets/images/0-Reference-Brain/Reference-Brain-{}.jpg'.format(i)
    io.imsave(img_path, image)

(1, 138, 1, 621, 1406)
(138, 621, 1406)


### Create database entries for exported images

#### Connect to Postgresql db

In [61]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

#### Initialize '*images*' table

In [62]:
create_table_command = (
"""
    DROP TABLE images;
    CREATE TABLE images (
        image_id SERIAL PRIMARY KEY,
        line_name TEXT NOT NULL,
        image_path TEXT NOT NULL
    );
""")

In [63]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

#### Iterate stack directories and create database entries for each JPG

In [65]:
stacks = sorted([d for d in os.listdir('../app/assets/images/0-Lines') if not d.startswith('.')])
stacks

['0-Reference-Brain',
 '6.7FRhcrtR-Gal4-uasKaede',
 'Anti-5HT',
 'Anti-Gad67',
 'Anti-GlyR',
 'Anti-TH',
 'Anti-Zn1',
 'Anti-Zn12(Hnk-1)',
 'Anti-Znp1(Synaptotagmin2)',
 'Anti-Zrf1(GFAP)',
 'Anti-Zrf2',
 'Anti-tERK',
 'Elavl3-GCaMP5G',
 'Elavl3-H2BRFP',
 'EtVmat2-GFP',
 'Gad1b-GFP',
 'Glyt2-GFP',
 'Hcrt-RFP',
 'Isl1-GFP',
 'Isl2bGal4-uasDendra',
 'MH_13319-Gal4',
 'MH_15576-Gal4',
 'MH_16659-Gal4',
 'MH_16715-Gal4',
 'MH_16913-Gal4',
 'MH_16914-Gal4',
 'MH_16915-Gal4',
 'MH_16916-Gal4',
 'MH_16946-Gal4',
 'MH_17056-Gal4',
 'MH_17767-Gal4',
 'MH_17768-Gal4',
 'MH_17776-Gal4',
 'MH_17777-Gal4',
 'MH_17787-Gal4',
 'MH_17845-Gal4',
 'MH_17846-Gal4',
 'MH_18107-Gal4',
 'MH_18108-Gal4',
 'MH_18327-Gal4',
 'MH_18329-Gal4',
 'MH_19322-Gal4',
 'MH_19566-Gal4',
 'MH_20360-Gal4',
 'MH_20361-Gal4',
 'MH_20454-Gal4',
 'MH_20552-Gal4',
 'MH_20553-Gal4',
 'MH_20638-Gal4',
 'MH_20639-Gal4',
 'MH_20640-Gal4',
 'MH_20641-Gal4',
 'MH_20642-Gal4',
 'MH_20643-Gal4',
 'MH_20644-Gal4',
 'MH_20646-Gal4',
 'MH

In [66]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [67]:
counter = 0
for stack_name in stacks:
    folder_path = '../app/assets/images/0-Lines/%s/' % stack_name
    rel_folder_path = '%s/' % stack_name
    stack = natsorted([i for i in os.listdir(folder_path) if not i.startswith('.')])
    
    for i, s in enumerate(stack):
        rel_img_path = rel_folder_path + s
        insert_command = ("INSERT INTO images (image_id, line_name, image_path) values (%i, '%s', '%s')" 
                % (counter, stack_name, rel_img_path))
        
        try:
            insert_command
            cursor.execute(insert_command)
            counter += 1
        except Exception as e:
            print(e)

#### Check that appropriate number rows, and appropriate row content

In [68]:
cursor.execute('SELECT * FROM images')
rows = cursor.rowcount
print(rows)

25806


In [69]:
cursor.close()
conn.commit()

In [70]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [71]:
cursor.execute('SELECT * FROM images order by image_id')

In [72]:
rows = cursor.fetchall()
for row in rows[100:300]:
    print(row)

(100, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-100.jpg')
(101, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-101.jpg')
(102, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-102.jpg')
(103, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-103.jpg')
(104, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-104.jpg')
(105, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-105.jpg')
(106, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-106.jpg')
(107, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-107.jpg')
(108, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-108.jpg')
(109, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-109.jpg')
(110, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-110.jpg')
(111, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-111.jpg')
(112, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-112.jpg')
(113, '0-Reference-Brain', '0-Reference-Brain/Reference-Brain-11